In [ ]:
from qat.lang.AQASM import Program, H, CNOT, AbstractGate
import numpy as np

In [ ]:
import sys
sys.version

In [ ]:
from helpers import U_ZZ, U_XX, U_YY, U_IZ, U_ZI, U_II

In [ ]:
prog = Program()

n = 2
t = 6

q = prog.qalloc(n+t)

prog.apply(U_ZZ(4.5).ctrl(), [q[0]]+q[t:])
prog.apply(U_XX(2.85).ctrl(), [q[1]]+q[t:])
prog.apply(U_YY(78.8).ctrl(), [q[2]]+q[t:])
prog.apply(U_IZ(78.8).ctrl(), [q[3]]+q[t:])
prog.apply(U_ZI(78.8).ctrl(), [q[4]]+q[t:])
prog.apply(U_II(78.8).ctrl(), [q[5]]+q[t:])


In [ ]:
circ = prog.to_circ(inline=True)

%qatdisplay --svg circ

In [ ]:
import json

with open('hamiltonian_data.json','r') as f:
    l = json.load(f)

In [ ]:
print(l[0].keys())

In [ ]:
from helpers import pea

In [ ]:
t = 10

prog = Program()

qbits = prog.qalloc(t+2) 

prog.apply(pea(l[0],4,t,0.1), qbits)

circ = prog.to_circ()

%qatdisplay --svg circ 

In [ ]:
from qat.qpus import get_default_qpu
qpu = get_default_qpu()

In [ ]:
job = circ.to_job(qubits = range(t), lsb_first=True)

In [ ]:
result = qpu.submit(job)
p = -1

result.lsb_first = False
for sample in result:
    if sample.probability > p:
        state = sample.state
        p = sample.probability
        sam = sample

print(state, p)
print(sam._state)
print(sam)

print(bin(sam._state))
for b in state:
    print(b)

In [ ]:
import matplotlib.pylab as plt
energy = []
Rs = []

t = 16
p = 1

for coeffs in l[::2]:
    Rs.append(coeffs["R"])

    prog = Program()

    qbits = prog.qalloc(t+2) 

    prog.apply(pea(coeffs,p,t,0.3), qbits)

    circ = prog.to_circ()
    job = circ.to_job(qubits = range(t))

    result = qpu.submit(job)

    prob = -1
    

    for sample in result:
        if sample.probability > prob:
            state = sample.state
            prob = sample.probability
            sam = sample
    
    print(state)
    energy.append(sam._state)
    


In [ ]:
plt.plot(Rs, energy)
plt.ylim([0,1000])
    